In [1]:
# Notebook to save the time profile of events for reading into simulatoin
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'svg'
import math 

In [2]:
Mode = ["DEMOpp","DEMOpp"]

# data = pd.read_csv(f"../Files/{Mode[0]}/EventInfo_merge_{Mode[1]}.csv", delimiter = ",", names = ["event", "x", "y", "z", "t"])
data = pd.read_csv(f"../Files/{Mode[0]}/EventInfo_DEMO_10bar.csv", delimiter = ",", names = ["event", "x", "y", "z", "t"])

# Cut the first bin
data = data[data['t'] > 200]
data = data.reset_index(drop=True)
display(data)

,event,x,y,z,t
0,21,0.325,1.260,0.483,3423.18
1,21,0.332,1.253,0.475,3475.48
2,21,0.331,1.253,0.473,3480.11
3,21,0.330,1.255,0.462,3533.95
4,21,0.329,1.254,0.460,3539.67
...,...,...,...,...,...
25276718,19609,0.353,0.602,-0.498,6763.64
25276719,19609,0.353,0.601,-0.498,6764.04
25276720,19609,0.353,0.601,-0.498,6764.35
25276721,19609,0.353,0.601,-0.499,6764.71


In [3]:

# Here we subtract off the first time bin to shift all the EL values to zero

def GetSubDf(data, var, var_sub, query):
    data_copy = data.copy(deep=True)

    data_sub = data_copy.drop_duplicates(subset= ["event"], keep='first')
    data_sub = data_sub.drop(columns = query)

    data_copy['Count'] = 1
    data_copy = data_copy.groupby(["event"]).Count.count().reset_index()

    # get a dataframe with the first time row to subtract from
    data_merge = pd.DataFrame()
    data_sub = data_sub.merge(data_copy, on='event', how="inner")
    data_sub = data_sub.reset_index(drop = True)
    data_sub = data_sub.loc[data_sub.index.repeat(data_sub.Count)]
    data_sub = data_sub.drop(columns = ["Count"])
    data_sub = data_sub.rename(columns={var: var_sub})
    data_sub = data_sub.reset_index(drop=True)
    return data_sub



In [4]:

data_sub = pd.DataFrame()
query = ["x", "y", "z"]
data_sub = GetSubDf(data, "t", "t_sub", query)
data["t"] = data["t"] - data_sub["t_sub"]

data_sub = pd.DataFrame()
query = ["t", "y", "z"]
data_sub = GetSubDf(data, "x", "x_sub", query)
data["x"] = data["x"] - data_sub["x_sub"]

data_sub = pd.DataFrame()
query = ["x", "t", "z"]
data_sub = GetSubDf(data, "y", "y_sub", query)
data["y"] = data["y"] - data_sub["y_sub"]

data_sub = pd.DataFrame()
query = ["x", "y", "t"]
data_sub = GetSubDf(data, "z", "z_sub", query)
data["z"] = data["z"] - data_sub["z_sub"]

display(data)


,event,x,y,z,t
0,21,0.000,0.000,0.000,0.00
1,21,0.007,-0.007,-0.008,52.30
2,21,0.006,-0.007,-0.010,56.93
3,21,0.005,-0.005,-0.021,110.77
4,21,0.004,-0.006,-0.023,116.49
...,...,...,...,...,...
25276718,19609,-0.042,0.046,-0.996,3280.34
25276719,19609,-0.042,0.045,-0.996,3280.74
25276720,19609,-0.042,0.045,-0.996,3281.05
25276721,19609,-0.042,0.045,-0.997,3281.41


In [8]:
# Export the dataframe 
# data = data[ data["event"] < 5000]

# Convert some columns to mm to save space
data.to_csv("../Files/DEMOpp/DEMO_Profiles.csv", index=False, header=False, float_format='%4.2f')